#**Trabalho final - Classificador de produtos com GenAI**

**Nota de atenção:**
- Leia com atenção o descritivo do trabalho e as orientações do template.
- O trabalho deve ser entregue **respeitando a estrutura do arquivo de template**, utilizando o notebook "Template Trabalho final - Classificador de produtos com GenAI.ipynb" e compactado no formato .zip.
- Deve haver apenas um arquivo no formato .ipynb, consolidando todo o trabalho.

**Participantes (RM - NOME):**<br>

361485 - GUSTAVO QUEIROZ RIBEIRO<br>


###**Caso de uso:**

Uma empresa de marketplace, disponibiliza sua plataforma para diversos vendedores cadastrarem seus produtos em diferentes categorias previamente definidas. Essas categorias são utilizadas para melhor distribuir e divulgar seus produtos para os clientes e usuários da plataforma. Mas nem todos os vendedores respeitam essas categorias, regras e as diretrizes do marketplace.

Pense nos diversos problemas que podemos enfrentar:
- Vendedores que cadastram produtos em categorias erradas;
- Vendedores que querem vender produtos ilícitos;
- Vendedores que querem vender produtos que não são permitidos pelas políticas do marketplace e por aí vai...

**Será que é possível validar o cadastro desses produtos e categorias, produto por produto**? Um por um?... Que trampo, não???

###**Desafio:**
- Conseguimos ajudar a mitigar parte desse problema?
- Você conseguiria criar algum **processo** que diminua esse trabalho manual e que **valide a categoria** desses produtos?

Bom, podemos criar um **processo que seja capaz de classificar um produto através do nome e da descrição**, e depois podemos confrontar com a categoria e premissas da plataforma usando **IA Generativa e modelos LLMs**.

###**Orientações:**

---
####**Usem o Google Colab com Python e esse template para desenvolverem o trabalho.**
---

**1. Análise exploratória**, vamos começar explorando a base de dados de produtos [1]?
  - Faça uma análise exploratória para entender os dados e estrutura do dataframe.
  - Crie uma nova coluna no dataframe chamada **`texto`** concatenando as colunas **`nome`** e **`descricao`**. Ex.: `nome + " " + descricao`.
  - Crie um dataframe novo e selecionando uma amostra aleatória de **100 registros** usando o `random_state = 42` (veja o exemplo abaixo).
  
  ***Obs**.:
    - É importante manter essa configuração da amostra para os trabalhos ficarem comparáveis e classificando os mesmos produtos.
    - Faça a análise exploratória com a base completa e não apenas com o sample.

**2. Desenvolvimento e testes**, nessa parte é onde vocês podem explorar o desenvolvimento do trabalho aplicando as técnicas, ferramentas e serviços de GenAI.  
  - Explore diferentes formas de tratar o problema. Comece selecionando casos individuais para testar as ferramentas, framework, APIs e técnicas de prompt engineering.
  - Fiquem à vontade para explorar os serviços e frameworks vistos em aula: API da OpenAI Platform, API da Azure AI Foundry e LangChain.
  - Sejam criativo, mas não precisa de complexidade e podem explorar outras formas de desenvolver.
  - Explique as decisões e racional do desenvolvimento. Abuse dos comentários.

**3. Processo final**, aqui nessa parte separe apenas o processo final com um pipeline completo para classificar a categoria dos produtos.
  - Crie uma função de receba um `texto` (nome + descrição do produto) e retorne a categoria classificada pelo texto.
  - Aplique essa função no dataframe com os 100 casos.
  - Resultado esperado: No dataframe de produto com os 100 casos, crie uma nova coluna de categoria: `categoria_genai`.
  - Valide se a classificação da Ia Generativa de cada produto está correta ou divergente comparando com a coluna `categora` escolhida pelo vendedor. Crie uma nova coluna `validacao_categoria` com os valores da validação: `correta` ou `divergente`.

###**Avaliação:**
O trabalho será avaliado pelas seguintes diretrizes:
  - Demonstração de conhecimento com os temas abordados em sala de aula.
  - Utilização correta dos frameworks, APIs e aplicação das técnicas de prompt engineering.
  - Organização, comentários e explicação certamente vão ajudar na nota.
  - Resultado esperado seguindo as orientações do professor nesse template.

###**Atenção:**
- Use sua conta da **Azure AI Foundry** ou da **OpenAI Platform** para desenvolver o trabalho, mas dê preferência para a conta da Azure por causa dos limites de crédito. **Não deixe suas credenciais no trabalho, por favor!**
- Trabalhos iguais são passíveis de reprovação ou desconto de nota.
- Respeite a estrutura do template fornecido pelo professor.
- Limite de 4 pessoas por grupo, de preferência o mesmo grupo do Startup One.

###**[1] Base de dados de produtos - https://raw.githubusercontent.com/queirozene/generative-ai/refs/heads/main/data/produtos.csv**


##**1. Análise exploratória**

Desenvolva aqui:

In [ ]:
# Desenvolvimento da análise exploratória da base completa
import pandas as pd

df_exp = pd.read_csv("https://raw.githubusercontent.com/queirozene/generative-ai/refs/heads/main/data/produtos.csv", delimiter=";", encoding='utf-8')

In [ ]:
df_exp.info()

Análise: DF gerado tipo pandas.DataFrame e três colunas do tipo object classficados como string.

Colunas presentes:

- nome = contém o nome do produto, podendo conter mais informações como envio, modo de venda etc.

- descricao = contém a descrição sobre o produto e informações complementares de venda para alguns casos, existem produtos que não possui descrição.

- categoria = categoria do produto cadastrada préviamente pelo vendedor

In [ ]:
df_exp.describe()

In [ ]:
df_exp['categoria'].unique()

Análise: Base completa possui 4080 registros e nem todos são produtos únicos. Também existem produtos sem descrição e temos presentes 4 categorias diferentes classificadas pelos vendedores (brinquedo, maquiagem, game e livro)

Agora vamos criar o dataframe com 100 registros de exemplo e realizar um tratamento melhor nos nossos dados!

In [ ]:
import pandas as pd

# Dataframe novo selecionando uma amostra aleatória de 100 registros usando o random_state = 42
df_dev = pd.read_csv("https://raw.githubusercontent.com/queirozene/generative-ai/refs/heads/main/data/produtos.csv", delimiter=";", encoding='utf-8').sample(100, random_state=42)

# Tratamento na descrição: para não corrermos o risco possuirmos linhas com a coluna vazia e isso atrapalhar na análise
df_dev['descricao'] = df_dev['descricao'].fillna('Não possui descrição.')

In [ ]:
# Criação da coluna 'texto' que concatena <nome do produto | descricao produto>
df_dev['texto'] = df_dev['nome'] + " | " + df_dev['descricao']

# Criação de coluna ID para controle da amostra
df_dev['id_produto'] = range(1, len(df_dev) + 1)

df_dev.head()

In [ ]:
# Criação do DF para testes de desenvolvimento
df_stage = df_dev.head(10).copy()

##**2. Desenvolvimento e testes**

Desenvolva aqui:

Download da lib do LangChain:

https://python.langchain.com/api_reference/reference.html

In [ ]:
!pip install langchain langchain-openai --quiet

In [ ]:
from langchain_openai import AzureChatOpenAI
from pydantic import Field, BaseModel
from langchain_core.output_parsers import JsonOutputParser

subscription_key = ""# @param {"type":"string"}
azure_endpoint=""# @param {"type":"string"}
api_version=""# @param {"type":"string"}
deployment = ""# @param {"type":"string"}

# Criação do llm
llm = AzureChatOpenAI(
    api_key = subscription_key,
    azure_endpoint = azure_endpoint,
    api_version = api_version,
    azure_deployment = deployment,
    temperature = 0.0,
)

# Teste da llm, para ver se tudo foi criado corretamente kkkkkk
response = llm.invoke("Convença meu professor de GenAI me dar 10 neste trabalho!")
print(response.text())

In [ ]:
# Uso do pydantic para formatar a saída com o JsonOutputParser
class Saida(BaseModel):
  id_produto: int = Field(..., description="Id do produto")
  categoria_genai: str = Field(..., description="Categoria preditada pela GenAI")

# Criação do parseador para a llm
parser = JsonOutputParser(pydantic_object=Saida)

In [ ]:
# Criação do prompt do sistema - system

system = """
# SUA FUNÇÃO:

Você trabalhará como um classificador de produtos. E deve ser direto lendo o **texto** (concatenação do nome do produto | descrição, separados por '|') de cada item e classificar dentre essas categorias presentes:
- Livro;
- Brinquedo;
- Maquiagem;
- Game;
- Outra Categoria - <sugestão> (neste caso você deve sugerir uma nova categoria que deve ser criada, apenas se necessário)

# CATEGORIAS E SUAS CARACTERÍSTICAS:

- **Livro**: produtos de leitura, físicos ou digitais.
  Exemplos: romance, quadrinhos, livro didático, apostila, e-book.
  Palavras-chave comuns: "autor", "edição", "capa", "volume", "coleção".

- **Brinquedo**: itens voltados para diversão ou lazer, especialmente infantil.
  Exemplos: boneca, carrinho, jogo de tabuleiro, lego, pelúcia.
  Palavras-chave comuns: "criança", "diversão", "brincar", "educativo".

- **Maquiagem**: produtos cosméticos usados na face, pele, unhas ou cabelo para embelezar.
  Exemplos: batom, base, rímel, sombra, esmalte, pincel de maquiagem.
  Palavras-chave comuns: "cor", "tom", "pele", "make", "cosmético".

- **Game**: jogos eletrônicos ou acessórios relacionados.
  Exemplos: jogos de PlayStation, Xbox, PC, Nintendo; controles, headsets gamer.
  Palavras-chave comuns: "PS5", "Xbox", "Steam", "gamer", "console".

- **Outra Categoria - <sugestão>**: se o produto não se encaixar em nenhuma das anteriores, você deve criar uma nova categoria coerente.
  Exemplos: "Notebook Dell 16GB RAM" → `Outra Categoria - Informática`.
  Exemplos: "Smartphone Samsung Galaxy" → `Outra Categoria - Celulares`.

OBS: Se atente a REAL UTILIDADE do produto na hora de categorizar.

# INSTRUÇÕES IMPORTANTES:

1. Seja **direto e consistente**: classifique sempre em uma única categoria.
2. Leia todo o texto (nome + descrição), mas **priorize palavras-chave relevantes**.
3. Se não houver informações suficientes, use sua melhor inferência e crie uma categoria sugerida.
4. Se um produto se encaixar nas categorias existentes, NÃO realize a criação de uma nova, encaixe na existente.

# FORMATAÇÃO DE ENTRADA:

- Você receberá: 'ID: <id_produto> TEXTO: <concatenação do nome do produto | descrição, separados por '|'>

# FORMATAÇÃO DE SAÍDA:

- Sempre responda no formato JSON válido e respeite o nome dos campos:

{formatacao_saida}
"""

# Criação do prompt do usuário, informações do DF
prompt = """
ID: {id_produto}
TEXTO: {texto}
"""

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Configuração do modelo de template e parseamento de saída
classificacao_model = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", prompt)]).partial(formatacao_saida=parser.get_format_instructions())

# Aplicação da técnica de LCEL (LangChain Expression Language)
chain_classificacao = classificacao_model | llm | parser

In [ ]:
# Teste de entrada: personalize como quiser o id_produto e texto, para verificar o funcionamento :)
teste_entrada = chain_classificacao.invoke({
    "id_produto": 123,
    "texto": "Iphone 17 | O melhor da apple."
})
print(teste_entrada)

### Cometario do método adotado para a função Python:

Para aplicação ao meu DF irei utilizar o método que encontrei estudando a documentação chamada batch() onde ela faz a implementação de execução de invokes de forma paralela utilizando thread pool executor segundo a documentação, totalmente compatível com a chain do tipo `RunnableSequence`:

[batch()](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate.batch)

In [ ]:
import math
from tqdm import tqdm

def classificador_batch(df, chain, id_col="id_produto", texto_col="texto", batch_size=5):

  # Separa as informações de entrada para a iteração e define o total de batches + prepara dicionario de saida
  entradas = df[[id_col, texto_col]].to_dict(orient="records")
  total_batches = math.ceil(len(entradas) / batch_size)
  retorno = []

  print(f"Classificando {len(entradas)} produtos em {total_batches} lotes (batch_size={batch_size})...\n")

  # Realiza a iteração 'for' para a criação dos lotes que alimenta o método batch() e adiciona gradativamente a lista de retorno
  for i in tqdm(range(0, len(entradas), batch_size)):
    lote = entradas[i:i + batch_size]

    try:
      results = chain.batch(lote)
      retorno.extend(results)
    except Exception as e:
      print(f"Erro no lote {i//batch_size + 1}: {e}")

      for item in lote:
        retorno.append({
            "id_produto": item[id_col],
            "cateogoria_genai": f"Erro: {e}"})

  # Cria o dataframe com a lista de retorno com os id_produto e as categorias geradas pela GenAI
  df_result = pd.DataFrame(retorno)

  # Realiza o merge do df original recebido na função e o df_result após a classificação da GenAI, levando a coluna categoria_genai
  df_merge = df.merge(df_result, on=id_col, how="left")

  print("Classificação concluída!!")

  return df_merge

In [ ]:
df_classificacao = classificador_batch(df=df_stage, chain=chain_classificacao)

In [ ]:
df_classificacao.tail()

In [ ]:
def validar_classificador(df, col_vendedor="categoria", col_genai="categoria_genai"):

  # Aqui eu crio uma cópia do df recebido só para não ter risco de tomar warning ou erro do pandas por ter manipulação no df em passos anteriores
  df_validacao = df.copy()

  # Padroniza para lower, para comparação porque python é bem case-sensitive
  validation = df_validacao[col_vendedor].str.lower() == df_validacao[col_genai].str.lower()

  df_validacao["validacao_categoria"] = validation.map({True: "Correta", False: "Divergente"})

  # Criação de um resumo simples
  total = len(df_validacao) #total de linhas no df
  corretas = (df_validacao["validacao_categoria"] == "Correta").sum() # Total de linhas que a categoria do vendedor está correta
  incorretas = (df_validacao["validacao_categoria"] == "Divergente").sum() # Total de linhas que a categoria do vendedor está incorreta
  taxa_acerto = round((corretas / total) * 100, 2) if total > 0 else 0 # Porcentagem de acerto do cadastro de categorias feitas pelo vendedor

  print(f"Total: {total} | Corretas: {corretas} | Incorretas: {incorretas} | Taxa de acerto dos lojistas: {taxa_acerto}%")

  return df_validacao

In [ ]:
df_validacao = validar_classificador(df=df_classificacao)

df_validacao.head()

In [ ]:
df_validacao["categoria_genai"].value_counts()

Teste de junção das duas funções para ficar em um único pipeline para o final

In [ ]:
def classificador_batch_and_validation(df, chain, id_col="id_produto", texto_col="texto", col_vendedor="categoria", batch_size=5):

  def validar_classificador(df_classificado):

    validation = df_classificado[col_vendedor].str.lower() == df_classificado["categoria_genai"].str.lower()
    df_classificado["validacao_categoria"] = validation.map({True: "Correta", False: "Divergente"})

    total = len(df_classificado)
    corretas = (df_classificado["validacao_categoria"] == "Correta").sum()
    incorretas = (df_classificado["validacao_categoria"] == "Divergente").sum()
    taxa_acerto = round((corretas / total) * 100, 2) if total > 0 else 0

    print(f"Total: {total} | Corretas: {corretas} | Incorretas: {incorretas} | Taxa de acerto dos lojistas: {taxa_acerto}%")

    return df_classificado

  entradas = df[[id_col, texto_col]].to_dict(orient="records")
  total_batches = math.ceil(len(entradas) / batch_size)
  retorno = []

  print(f"Classificando {len(entradas)} produtos em {total_batches} lotes (batch_size={batch_size})...\n")

  for i in tqdm(range(0, len(entradas), batch_size)):
    lote = entradas[i:i + batch_size]

    try:
      results = chain.batch(lote)
      retorno.extend(results)
    except Exception as e:
      print(f"Erro no lote {i//batch_size + 1}: {e}")

      for item in lote:
        retorno.append({
            "id_produto": item[id_col],
            "cateogoria_genai": f"Erro: {e}"})

  df_result = pd.DataFrame(retorno)
  df_merge = df.merge(df_result, on=id_col, how="left")

  print("Classificação concluída!!")

  df_final = validar_classificador(df_classificado=df_merge)

  return df_final

In [ ]:
df_final = classificador_batch_and_validation(df_stage, chain_classificacao, batch_size=2)

In [ ]:
df_final.head()

##**3. Processo final**

Desenvolva aqui:

In [ ]:
!pip install langchain langchain-openai --quiet

### Import das libs necessárias

In [ ]:
import pandas as pd
from langchain_openai import AzureChatOpenAI
from pydantic import Field, BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
import math
from tqdm import tqdm

### Criação do df_dev, com os padrões solicitados e tratamentos adicionais:

In [ ]:
df_dev = pd.read_csv("https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv",
                     delimiter=";",
                     encoding='utf-8').sample(100, random_state=42)

df_dev['descricao'] = df_dev['descricao'].fillna('Não possui descrição.')
df_dev['texto'] = df_dev['nome'] + " | " + df_dev['descricao']
df_dev['id_produto'] = range(1, len(df_dev) + 1)

### Criação da llm via AzureChatOpenAI

In [ ]:
subscription_key = ""# @param {"type":"string"}
azure_endpoint=""# @param {"type":"string"}
api_version=""# @param {"type":"string"}
deployment = ""# @param {"type":"string"}

llm = AzureChatOpenAI(api_key = subscription_key,
                      azure_endpoint = azure_endpoint,
                      api_version = api_version,
                      azure_deployment = deployment,
                      temperature = 0.0,)

### Estrutura função completa

In [ ]:
class Saida(BaseModel):
  id_produto: int = Field(..., description="Id do produto")
  categoria_genai: str = Field(..., description="Categoria preditada pela GenAI")

parser = JsonOutputParser(pydantic_object=Saida)

system = """
# SUA FUNÇÃO:

Você trabalhará como um classificador de produtos. E deve ser direto lendo o **texto** (concatenação do nome do produto | descrição, separados por '|') de cada item e classificar dentre essas categorias presentes:
- Livro;
- Brinquedo;
- Maquiagem;
- Game;
- Outra Categoria - <sugestão> (neste caso você deve sugerir uma nova categoria que deve ser criada, apenas se necessário)

# CATEGORIAS E SUAS CARACTERÍSTICAS:

- **Livro**: produtos de leitura, físicos ou digitais.
  Exemplos: romance, quadrinhos, livro didático, apostila, e-book.
  Palavras-chave comuns: "autor", "edição", "capa", "volume", "coleção".

- **Brinquedo**: itens voltados para diversão ou lazer, especialmente infantil.
  Exemplos: boneca, carrinho, jogo de tabuleiro, lego, pelúcia.
  Palavras-chave comuns: "criança", "diversão", "brincar", "educativo".

- **Maquiagem**: produtos cosméticos usados na face, pele, unhas ou cabelo para embelezar.
  Exemplos: batom, base, rímel, sombra, esmalte, pincel de maquiagem.
  Palavras-chave comuns: "cor", "tom", "pele", "make", "cosmético".

- **Game**: jogos eletrônicos ou acessórios relacionados.
  Exemplos: jogos de PlayStation, Xbox, PC, Nintendo; controles, headsets gamer.
  Palavras-chave comuns: "PS5", "Xbox", "Steam", "gamer", "console".

- **Outra Categoria - <sugestão>**: se o produto não se encaixar em nenhuma das anteriores, você deve criar uma nova categoria coerente.
  Exemplos: "Notebook Dell 16GB RAM" → `Outra Categoria - Informática`.
  Exemplos: "Smartphone Samsung Galaxy" → `Outra Categoria - Celulares`.

OBS: Se atente a REAL UTILIDADE do produto na hora de categorizar.

# INSTRUÇÕES IMPORTANTES:

1. Seja **direto e consistente**: classifique sempre em uma única categoria.
2. Leia todo o texto (nome + descrição), mas **priorize palavras-chave relevantes**.
3. Se não houver informações suficientes, use sua melhor inferência e crie uma categoria sugerida.
4. Se um produto se encaixar nas categorias existentes, NÃO realize a criação de uma nova, encaixe na existente.

# FORMATAÇÃO DE ENTRADA:

- Você receberá: 'ID: <id_produto> TEXTO: <concatenação do nome do produto | descrição, separados por '|'>

# FORMATAÇÃO DE SAÍDA:

- Sempre responda no formato JSON válido e respeite o nome dos campos:

{formatacao_saida}
"""

prompt = """
ID: {id_produto}
TEXTO: {texto}
"""

classificacao_model = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", prompt)]).partial(formatacao_saida=parser.get_format_instructions())

chain_classificacao = classificacao_model | llm | parser


In [ ]:
def classificador_batch_and_validation(df, chain, id_col="id_produto", texto_col="texto", col_vendedor="categoria", batch_size=5):

  def validar_classificador(df_classificado):

    validation = df_classificado[col_vendedor].str.lower() == df_classificado["categoria_genai"].str.lower()
    df_classificado["validacao_categoria"] = validation.map({True: "Correta", False: "Divergente"})

    total = len(df_classificado)
    corretas = (df_classificado["validacao_categoria"] == "Correta").sum()
    incorretas = (df_classificado["validacao_categoria"] == "Divergente").sum()
    taxa_acerto = round((corretas / total) * 100, 2) if total > 0 else 0

    print(f"Total: {total} | Corretas: {corretas} | Incorretas: {incorretas} | Taxa de acerto dos lojistas: {taxa_acerto}%")

    return df_classificado

  entradas = df[[id_col, texto_col]].to_dict(orient="records")
  total_batches = math.ceil(len(entradas) / batch_size)
  retorno = []

  print(f"Classificando {len(entradas)} produtos em {total_batches} lotes (batch_size={batch_size})...\n")

  for i in tqdm(range(0, len(entradas), batch_size)):
    lote = entradas[i:i + batch_size]

    try:
      results = chain.batch(lote)
      retorno.extend(results)
    except Exception as e:
      print(f"Erro no lote {i//batch_size + 1}: {e}")

      for item in lote:
        retorno.append({
            "id_produto": item[id_col],
            "cateogoria_genai": f"Erro: {e}"})

  df_result = pd.DataFrame(retorno)
  df_merge = df.merge(df_result, on=id_col, how="left")

  print("Classificação concluída!!")

  df_final = validar_classificador(df_classificado=df_merge)

  return df_final

In [ ]:
df_final = classificador_batch_and_validation(df=df_dev,
                                              chain=chain_classificacao,
                                              batch_size=10) # Pode ajustar aqui quantas linhas quer em cada lote do .batch() 10 = 10 lotes / 50 = 2 lotes

In [ ]:
df_final["categoria_genai"].value_counts()

In [ ]:
df_final["validacao_categoria"].value_counts()

In [ ]:
df_final.to_csv("df_final_GustavoQueirozRibeiro_RM361485",
                sep=";",
                index=False,
                encoding="utf-8-sig")

In [ ]:
df_final.head()